In [134]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
intersection = pd.read_csv("./intersection.csv")

In [12]:
# convert df to gdf
from shapely import wkt

intersection['geometry'] = gpd.GeoSeries.from_wkt(intersection['geometry'])

In [13]:
intersection = gpd.GeoDataFrame(intersection, geometry=intersection['geometry'], crs='EPSG:32618')

In [17]:
# calculate area of each intersection
intersection['area'] = intersection.geometry.area

In [24]:
# aggregate the land cover in each vacant land
grouped_intersection =  intersection.groupby(['OBJECTID', 'gridcode'],)['area'].sum().reset_index()

In [39]:
intersection_wide = grouped_intersection.pivot(index='OBJECTID', columns='gridcode', values='area').fillna(0)

In [101]:
# rename the grid code
intersection_rename = intersection_wide.rename({1: 'tree canopy', 2: 'grass/shrub',3: 'bare earth',4: 'water',5: 'buildings',7: 'other paved surfaces'}, axis=1) 

(1) tree canopy, (2) grass/shrub, (3) bare earth, (4) water, (5) buildings, (6) roads, and (7) other paved surfaces

In [112]:
# merge vacant land data
vacant_land_na = pd.read_csv('./vacant_land_na.csv')
vacant_land_na['geometry'] = gpd.GeoSeries.from_wkt(vacant_land_na['geometry'])
vacant_land_na = gpd.GeoDataFrame(vacant_land_na, geometry=vacant_land_na['geometry'], crs='EPSG:32618')

In [113]:
# calculate the gross area of each vacant land
vacant_land_na['gross area'] = vacant_land_na.geometry.area

In [114]:
# merge the landcover and vacant land data
intersection_merge = pd.merge(intersection_rename,vacant_land_na,on = 'OBJECTID')

In [120]:
# calculate the proportion
intersection_merge['tree canopy %'] = intersection_merge['tree canopy']/intersection_merge['gross area']
intersection_merge['grass/shrub %'] = intersection_merge['grass/shrub']/intersection_merge['gross area']
intersection_merge['bare earth %'] = intersection_merge['bare earth']/intersection_merge['gross area']
intersection_merge['water %'] = intersection_merge['water']/intersection_merge['gross area']
intersection_merge['buildings %'] = intersection_merge['buildings']/intersection_merge['gross area']
intersection_merge['other paved surfaces %'] = intersection_merge['other paved surfaces']/intersection_merge['gross area']


In [125]:
cols = ['OBJECTID','tree canopy %','grass/shrub %','bare earth %','water %','buildings %','other paved surfaces %','median_t']

In [127]:
temp_cover = intersection_merge[cols].dropna()

In [129]:
temp_cover.head()

,OBJECTID,tree canopy %,grass/shrub %,bare earth %,water %,buildings %,other paved surfaces %,median_t
0,78,0.000000,0.171653,0.0,0.0,0.412690,0.415657,22.095707
1,150,0.184000,0.140671,0.0,0.0,0.583335,0.091994,28.064859
2,166,0.385852,0.142999,0.0,0.0,0.329685,0.141463,25.787609
3,185,0.771312,0.015409,0.0,0.0,0.196072,0.017207,25.520082
4,274,0.079999,0.023493,0.0,0.0,0.446899,0.449610,19.906439


Train a random forest

In [131]:
# from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import train_test_split

# Pipelines
from sklearn.pipeline import make_pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,OneHotEncoder
from sklearn.compose import ColumnTransformer

In [138]:
fea_cols = ['tree canopy %',
            'grass/shrub %',
            'bare earth %',
            'water %',
            'buildings %',
            'other paved surfaces %',
            'median_t']

In [132]:
# Do a 70/30 train/test split
train_set, test_set = train_test_split(temp_cover, test_size=0.3, random_state=42)

In [135]:
# Labels: log of sale prices
y_train = np.log(train_set["median_t"])
y_test = np.log(test_set["median_t"])

In [142]:
# Features
x_train = train_set[fea_cols] 
x_test = test_set[fea_cols]

In [139]:
# Set up the column transformer with a transformer
transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), fea_cols),
    ]
)

In [144]:
pipe = make_pipeline(
    transformer, RandomForestRegressor(random_state=42)
)

In [145]:
# Fit the training set
pipe.fit(x_train, y_train);
score = pipe.score(x_test, y_test);
print(f"Test Score = {score:0.4f}.")

Test Score = 0.9976.


In [146]:
# k-fold cross validation with GridSearchCV
from sklearn.model_selection import GridSearchCV
# create a pipeline
pipe_cv = make_pipeline(transformer, RandomForestRegressor(random_state=42))

model_step = "randomforestregressor"
param_grid = {
    f"{model_step}__n_estimators": [5, 10, 15, 20, 30, 50, 100, 200],
    f"{model_step}__max_depth": [2, 5, 7, 9, 13, 21, 33, 51],
}

# Create the grid and use 3-fold CV
grid = GridSearchCV(pipe_cv, param_grid, cv=3)

# Run the search
grid.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['tree '
                                                                          'canopy '
                                                                          '%',
                                                                          'grass/shrub '
                                                                          '%',
                                                                          'bare '
                                                                          'earth '
                                                                          '%',
                                                                          'water '
         

In [147]:
# The best estimator
grid.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['tree canopy %',
                                                   'grass/shrub %',
                                                   'bare earth %', 'water %',
                                                   'buildings %',
                                                   'other paved surfaces %',
                                                   'median_t'])])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=13, n_estimators=200,
                                       random_state=42))])

In [148]:
def evaluate(model, X_test, y_test):
    """
    Given a model and test features/targets, print out the 
    mean absolute error and accuracy
    """
    # Make the predictions
    predictions = model.predict(X_test)

    # Absolute error
    errors = abs(predictions - y_test)
    avg_error = np.mean(errors)

    # Mean absolute percentage error
    mape = 100 * np.mean(errors / y_test)

    # Accuracy
    accuracy = 100 - mape

    print("Model Performance")
    print(f"Average Absolute Error: {avg_error:0.4f}")
    print(f"Accuracy = {accuracy:0.2f}%.")

    return accuracy

In [149]:
# Evaluate the best random forest model
best_random = grid.best_estimator_
random_accuracy = evaluate(best_random,x_test, y_test)

Model Performance
Average Absolute Error: 0.0029
Accuracy = 99.91%.


In [150]:
best_score = best_random.score(x_test, y_test)
print(f"Test Score of the best model = {best_score:0.4f}.")

Test Score of the best model = 0.9976.


The model is best_random

In [151]:
# make predictions on the test set with the best_random model 
y_pred = best_random.predict(x_test)
y_pred = np.exp(y_pred)
# convert prediction outcome to a dataframe
test_pre = pd.DataFrame( { 'prediction': pd.Series(y_pred)} )

In [152]:
# merge the x_test and prediction outcome
x_test_index = x_test.reset_index(level=0)
prediction =x_test_index.merge(test_pre, left_index=True, right_index=True, indicator = True)
prediction_index = prediction.set_index('index')
prediction_index.head()

,tree canopy %,grass/shrub %,bare earth %,water %,buildings %,other paved surfaces %,median_t,prediction,_merge
index,,,,,,,,,
69,0.007514,0.020490,0.0,0.0,0.805968,0.166028,24.112165,24.076189,both
190,0.564034,0.369001,0.0,0.0,0.042996,0.023969,17.216916,17.176028,both
181,0.140164,0.127048,0.0,0.0,0.573295,0.159493,29.717943,29.780993,both
9,0.000000,0.260387,0.0,0.0,0.542058,0.197555,19.374901,19.351244,both
127,0.365624,0.022708,0.0,0.0,0.565003,0.046665,28.547405,28.639071,both
